In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130540

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,42440.9593
2018-02-28,41556.7474
2018-03-31,28889.0331
2018-04-30,32279.8543
2018-05-31,28439.3285
2018-06-30,38390.3305
2018-07-31,38856.3972
2018-08-31,42330.1823
2018-09-30,45899.6706


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    42440.9593
 2018-02-28    41556.7474
 2018-03-31    28889.0331
 2018-04-30    32279.8543
 2018-05-31    28439.3285
 2018-06-30    38390.3305
 2018-07-31    38856.3972
 2018-08-31    42330.1823
 2018-09-30    45899.6706
 2018-10-31    38744.3703
 2018-11-30    37261.8613
 2018-12-31    79366.5488
 2019-01-31    89145.5888
 2019-02-28    60707.2280
 2019-03-31    80195.2159
 2019-04-30    46966.1585
 2019-05-31    35666.9453
 2019-06-30    46334.4297
 2019-07-31    53305.8132
 2019-08-31    50771.1148
 2019-09-30    39621.6877
 2019-10-31    39214.7361
 2019-11-30    34533.1204
 2019-12-31    41993.3462
 2020-01-31    38960.6511
 2020-02-29    43628.5372
 2020-03-31    43818.9008
 2020-04-30    27574.9322
 2020-05-31    31242.1120
 2020-06-30    34240.6842
 2020-07-31    51476.0832
 2020-08-31    54164.7965
 2020-09-30    46556.5279
 2020-10-31    52334.2612
 2020-11-30    41312.6006
 2020-12-31    35159.6034
 2021-01-31    37032.6546
 2021-02-

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.127759
p-value : 0.024566
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=919.909, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=907.852, Time=0.14 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=908.933, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=901.821, Time=0.19 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.33 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=905.292, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=907.058, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=909.092, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=904.350, Time=0.10 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=907.024, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=909.025, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=907.868, Time=0.19 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.14 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept   : AIC

ARIMA(order=(0, 0, 3), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,0,3))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 3)   Log Likelihood                -445.862
Method:                       css-mle   S.D. of innovations           9328.115
Date:                Sat, 11 Sep 2021   AIC                            901.724
Time:                        04:39:16   BIC                            910.412
Sample:                             0   HQIC                           904.909
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.546e+04   4434.867     10.250      0.000    3.68e+04    5.41e+04
ma.L1.y        0.7574      0.289      2.619      0.009       0.191       1.324
ma.L2.y        0.5883      0.251      2.347      0.0

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([56246.21406914, 45455.53468197]), array([ 9328.11477942, 11701.54623091]), array([[37963.44505782, 74528.98308046],
       [22520.92550596, 68390.14385798]]))
